In [1]:
import pandas as pd

Import the data:

In [2]:
df = pd.read_csv('data.csv.zip', 
                 compression='zip', 
                 encoding='ISO-8859-1')

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
df.shape

(541909, 8)

In [5]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

Check the dataframe for duplicates:

In [6]:
df.duplicated().sum()

5268

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df.shape

(536641, 8)

***
The data contains records of both successful and cancelled transactions. If the user cancels the order, the transaction number (InvoiceNo) starts with C (canceled). 
How many orders have users cancelled?

In [9]:
df \
    .InvoiceNo \
    .str \
    .startswith('C') \
    .sum()

9251

So, users have cancelled `9251` orders.

***
Filter the data and left only rows with `Quantity > 0`:

In [10]:
df_sub = df.query('Quantity > 0')

In [11]:
df_sub.shape[0]

526054

***
Count the number of orders for each user from `Germany`. Leave only those who have made more than N transactions (InvoiceNo), where N - `80%` percent. Save these users id's.

In [12]:
df_german = df_sub \
                    .query('Country == "Germany"') \
                    .groupby('CustomerID') \
                    .agg({'InvoiceNo': pd.Series.nunique}) \
                    .rename(columns={'InvoiceNo': 'orders_num'})

In [13]:
df_german

,orders_num
CustomerID,
12426.0,1
12427.0,3
12468.0,2
12471.0,30
12472.0,7
...,...
13814.0,2
13815.0,4
13816.0,2


In [14]:
percentile_80 = df_german.orders_num.quantile(q=0.8)
percentile_80

7.0

In [15]:
top_german = df_german.query('orders_num > @percentile_80').index

In [16]:
top_german

Float64Index([12471.0, 12474.0, 12476.0, 12481.0, 12500.0, 12524.0, 12569.0,
              12600.0, 12619.0, 12621.0, 12626.0, 12647.0, 12662.0, 12705.0,
              12708.0, 12709.0, 12712.0, 12720.0],
             dtype='float64', name='CustomerID')

***
Filter initial dataframe using `top_german`:

In [17]:
df_top_german = df_sub.query('CustomerID in @top_german')

In [18]:
df_top_german

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,12/9/2011 11:59,4.95,12662.0,Germany
541727,581570,23077,DOUGHNUT LIP GLOSS,20,12/9/2011 11:59,1.25,12662.0,Germany
541728,581570,20750,RED RETROSPOT MINI CASES,2,12/9/2011 11:59,7.95,12662.0,Germany
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,12/9/2011 11:59,4.95,12662.0,Germany


***
Group dataframe `df_top_german` by `StockCode`. Which product was added to the basket most often, except `POST`?

In [19]:
df_top_german \
    .groupby('StockCode') \
    .agg({'InvoiceNo': pd.Series.nunique}) \
    .sort_values('InvoiceNo', ascending=False)

,InvoiceNo
StockCode,
POST,213
22326,52
22328,38
22423,34
20719,30
...,...
22563,1
22569,1
22574,1


Second most frequently ordered item with StockCode `22326`.

***
Create a `Revenue` column with the purchase amount:

In [20]:
df['Revenue'] = df['Quantity'].mul(df['UnitPrice'])

In [21]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


***
For each transaction (InvoiceNo), calculate the final order amount. Find the top 5 (InvoiceNo) by order amount

In [22]:
top_5_invoices = df \
                    .groupby('InvoiceNo') \
                    .agg({'Revenue': 'sum'}) \
                    .rename(columns={'Revenue': 'TotalRevenue'}) \
                    .sort_values('TotalRevenue', ascending=False) \
                    .head(5) 

In [23]:
top_5_invoices

,TotalRevenue
InvoiceNo,
581483,168469.60
541431,77183.60
574941,52940.94
576365,50653.91
556444,38970.00


In [24]:
top_5_invoices \
            .index \
            .str \
            .cat(sep=', ')

'581483, 541431, 574941, 576365, 556444'